# RESTful(表现层状态转移)
- Representational State Transfer
- 结构清晰
- 符合标准
- 易于理解
- 扩展方便
- URI-->资源
- 资源呈现出来的形式（txt、jpg）-->表现层
- 通过HTTP动词，对资源进行操作-->状态转化

# 核心
- 资源：url
- 形式：.mp4/.jpg/.png
- 状态：404、500
- HTTP动作：
- OPTION：查询支持的操作
- GET：获取资源
- POST：提交资源
- DELETE：删除资源
- PUT：修改资源（全部）
- PATCH：修改资源（部分）

# Django REST Framework
## Serialization序列化
- Serialization：将模型序列化为字典
- 之后字典可以转化为JSON等格式数据进行传输

---

```python
pip install django
pip install djangorestframework
pip install pygments # 代码高亮
```

---

```python
# serializers.py与models.py同级

from rest_framework import serializers
from snippets.models import Snippet, LANGUAGE_CHOICES, STYLE_CHOICES


class SnippetSerializer(serializers.Serializer):
    id = serializers.IntegerField(read_only=True)
    title = serializers.CharField(required=False, allow_blank=True, max_length=100)
    code = serializers.CharField(style={'base_template': 'textarea.html'})
    linenos = serializers.BooleanField(required=False)
    language = serializers.ChoiceField(choices=LANGUAGE_CHOICES, default='python')
    style = serializers.ChoiceField(choices=STYLE_CHOICES, default='friendly')

    def create(self, validated_data):
        """
        Create and return a new `Snippet` instance, given the validated data.
        """
        return Snippet.objects.create(**validated_data)

    def update(self, instance, validated_data):
        """
        Update and return an existing `Snippet` instance, given the validated data.
        """
        instance.title = validated_data.get('title', instance.title)
        instance.code = validated_data.get('code', instance.code)
        instance.linenos = validated_data.get('linenos', instance.linenos)
        instance.language = validated_data.get('language', instance.language)
        instance.style = validated_data.get('style', instance.style)
        instance.save()
        return instance
```

---

```python
# 上述代码可简化为
class SnippetSerializer(serializers.ModelSerializer):
    class Meta:
        model = Snippet # 指定模型类
        # fields = "__all__" # 指定所有
        # exclude = ("title") # 指定排除
        fields = ('id', 'title', 'code', 'linenos', 'language', 'style') # 指定序列化字段
```

---

```python
# views.py中使用

from django.http import HttpResponse, JsonResponse
from django.views.decorators.csrf import csrf_exempt
from rest_framework.renderers import JSONRenderer
from rest_framework.parsers import JSONParser
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer

@csrf_exempt
def snippet_detail(request, pk):
    """
    Retrieve, update or delete a code snippet.
    """
    try:
        snippet = Snippet.objects.get(pk=pk)
    except Snippet.DoesNotExist:
        return HttpResponse(status=404)

    if request.method == 'GET':
        serializer = SnippetSerializer(snippet)
        return JsonResponse(serializer.data)

    elif request.method == 'PUT':
        data = JSONParser().parse(request)
        serializer = SnippetSerializer(snippet, data=data)
        if serializer.is_valid():
            serializer.save()
            return JsonResponse(serializer.data)
        return JsonResponse(serializer.errors, status=400)

    elif request.method == 'DELETE':
        snippet.delete()
        return HttpResponse(status=204)

# urls.py添加路由
from django.conf.urls import url
from snippets import views

urlpatterns = [
    url(r'^snippets/$', views.snippet_list),
    url(r'^snippets/(?P<pk>[0-9]+)/$', views.snippet_detail),
]
        
# 项目添加路由
from django.conf.urls import url, include

urlpatterns = [
    url(r'^', include('snippets.urls')),
]
```

## 